In [ ]:
### import os
import os
import sys

## CHANGE THIS
dir2 = os.path.abspath("/volume/Orion/orion")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
import importlib

importlib.util.find_spec("oriÈon")
# import torch
import csv

import numpy as np
import pandas as pd
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 250)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 300)
pd.options.mode.chained_assignment = None  # default='warn'

def df_stats(df):
    from tabulate import tabulate 
    print("\n***** Shape: ", df.shape," *****\n")
    
    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()
    
    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(list_stat_val, columns=['Name', 'Null', 'Unique', 'Dtypes'])
    print(tabulate(df_stat_val, headers='keys', tablefmt='psql'))
    return df.head()

In [ ]:
df_predictions = pd.read_csv('../final_models/swin3d_s_5_32_2_RAdam_new_20231227-115517/df_predictions.csv')
df = pd.read_csv(
    "../data/train_val_test_LCA_REGRESSION_with_MHI_2021_data_NAS_path_mu.csv",
    sep="µ",
)

In [ ]:
merged_df = df.merge(df_predictions, left_on='FileName', right_on='filename')
display(merged_df)

In [ ]:
import numpy as np
merged_df['y_hat_sigmoid'] = 1 / (1 + np.exp(-merged_df['y_hat']))
correlation = merged_df['y_hat_sigmoid'].corr(merged_df['Value'])
print(correlation)
correlation = merged_df['y_hat'].corr(merged_df['Value'])
print(correlation)


In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(merged_df['y_true'], merged_df['y_hat_sigmoid'])
print(auc)


In [ ]:
display(df_stats(merged_df))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

### some analyses, we then dichotomized this continuous prediction into ≤/>40%, since LVEF ≤40% defines significant left ventricular dysfunction associated and has been a common cutoff for HFrEF.
### In sensitivity analysis, we also dichotomized using the <50%/≥50% cutoff which defines mildly reduced LVEF.5 More details in Supplement.

grouped_df = merged_df.groupby(['Patient_ID', 'y_true']).agg({'Value': 'mean', 'y_hat': 'mean', 'y_hat_sigmoid': 'mean'}).reset_index()
import numpy as np
grouped_df['y_hat_sigmoid'] = 1 / (1 + np.exp(-grouped_df['y_hat']))
correlation = grouped_df['y_hat_sigmoid'].corr(grouped_df['Value'])
print(correlation)
correlation = grouped_df['y_hat'].corr(grouped_df['Value'])
print(correlation)


from sklearn.metrics import roc_auc_score, confusion_matrix

grouped_df['y_true_40'] = np.where(grouped_df['Value'] <= 40, 1, 0)
auc = roc_auc_score(grouped_df['y_true'], grouped_df['y_hat_sigmoid'])
print(auc)
grouped_df['y_true_50'] = np.where(grouped_df['Value'] >= 50, 0, 1)
auc = roc_auc_score(grouped_df['y_true_50'], grouped_df['y_hat_sigmoid'])
print(auc)
grouped_df['y_hat_class'] = np.where(grouped_df['y_hat'] >= 0.26, 1, 0)

# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes, title):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(title)
    plt.show()



# Calculate and plot confusion matrix for y_true_40
confusion_matrix_40 = confusion_matrix(grouped_df['y_true_40'], grouped_df['y_hat_class'])
print("Confusion Matrix for y_true_40:")
print(confusion_matrix_40)
plot_confusion_matrix(confusion_matrix_40, classes=['0', '1'], title='Confusion Matrix for y_true_40')

# Calculate and plot confusion matrix for y_true_50
confusion_matrix_50 = confusion_matrix(grouped_df['y_true_50'], grouped_df['y_hat_class'])
print("Confusion Matrix for y_true_50:")
print(confusion_matrix_50)
plot_confusion_matrix(confusion_matrix_50, classes=['0', '1'], title='Confusion Matrix for y_true_50')